In [11]:
import pandas as pd
from deep_translator import (GoogleTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             single_detection,
                             batch_detection)
import nltk
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentAnalyzer

# Haalt info uit csv op.
dfDutch = pd.read_csv("GoogleReviewsZGVNoEmpty.csv")

dfSentiment = pd.DataFrame.from_dict({
    "id":[],
    "text": [],
    "compound": [],
    "neg":[],
    "neu":[],
    "pos":[],
})


index = 0

#Loop die er voor zorgt dat alle vertaalde reviews wordt uitgeprint
for col_name, review in dfDutch.iterrows():
 
    translated = GoogleTranslator(source='dutch', target='english').translate(text=dfDutch.loc[col_name]['review_text'])
    tokenized = sent_tokenize(translated)
    for line in tokenized:     
        sia = SentimentIntensityAnalyzer()
        ss = sia.polarity_scores(line)
        dfTemp = pd.DataFrame.from_dict({
                            "id":[col_name],
                            "text": [translated],
                            "compound": [ss["compound"]],
                            "neg":[ss["neg"]],
                            "neu":[ss["neu"]],
                            "pos":[ss["pos"]],
                            })
        index = index + 1
    dfSentiment = pd.concat([dfSentiment, dfTemp], ignore_index=True)
        #dfSentiment = dfSentiment.append({'id': index,'text': translated, 'compound': ss["compound"], 'neg': ss["neg"], 'neu': ss["neu"], 'pos': ss["pos"]}, ignore_index=True)

print(dfSentiment)


        id                                               text  compound  \
0      0.0  Before Corona, the phone was simply answered a...    0.0000   
1      1.0  Very sweet and caring staff in the mdl and sur...    0.0000   
2      2.0  Last Friday I called the hospital for my 2 yea...   -0.5697   
3      3.0  Costs if they call you is 60 euros per minute ...   -0.2382   
4      4.0  Let our son with a huge thick elbow sit for mo...    0.0000   
..     ...                                                ...       ...   
163  163.0  (Translated by Google) You can only dream of a...    0.8051   
164  164.0  (Translated by Google) I have not made any pro...    0.5106   
165  165.0  (Translated by Google) It was fine :). Thank y...    0.3612   
166  166.0  (Translated by Google) Very good hospital\n\n(...    0.7496   
167  167.0  (Translated by Google) Waiting hours\n\n(Origi...    0.0000   

       neg    neu    pos  
0    0.000  0.000  0.000  
1    0.000  1.000  0.000  
2    0.424  0.576 